<a href="https://colab.research.google.com/github/ahelikar/sensor_ai_water-purifier-project/blob/main/chatbot_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk #the nltk library is  imported for natural language processing

In [4]:
import os#importing neccessary libraries for readind texts and documents
import nltk
nltk.download('punkt_tab', quiet=True)

from nltk import word_tokenize #importing word tokenizer to convert string into tokens

file_content = "" #reading file content as a blank string

try:
    with open('water.txt', 'r') as f:
        file_content = f.read()
        print(file_content)
except FileNotFoundError:
    print("Error: water.txt not found. Please make sure the file exists in the correct directory.")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
word_file=word_tokenize(file_content) #Reading word file as tokens
print(word_file)

Water pollution (or aquatic pollution) is the contamination of water bodies, with a negative impact on their uses.[1]: 6  It is usually a result of human activities. Water bodies include lakes, rivers, oceans, aquifers, reservoirs and groundwater. Water pollution results when contaminants mix with these water bodies. Contaminants can come from one of four main sources. These are sewage discharges, industrial activities, agricultural activities, and urban runoff including stormwater.[2] Water pollution may affect either surface water or groundwater. This form of pollution can lead to many problems. One is the degradation of aquatic ecosystems. Another is spreading water-borne diseases when people use polluted water for drinking or irrigation.[3] Water pollution also reduces the ecosystem services such as drinking water provided by the water resource.

Sources of water pollution are either point sources or non-point sources.[4] Point sources have one identifiable cause, such as a storm d

In [ ]:
intents = [     # Declaring neccessary intents into a json file .More intents can be added
    'Greeting',
    'Goodbye',
    'Water Definition',
    'Water Coverage on Earth',
    'States of Water',
    'Water Cycle',
    'Importance of Clean Water',
    'Types of water bodies',
    'regulations of water bodies',
    'conservation',
    'pollution level',
    'pollutant',
    'toxins in water',
    'amount of dissolvable oxygen',
    'danger of marine and aqua life',
    'how useful the water is',
    'statistical analysis',
    'pH balance',
    'microplastic',
    'improving water quality',
    'hyperspectral remote sensing'

]

print("Identified Intents:")
for intent in intents:
    print(f"- {intent}")

Creating examples for intent

In [ ]:
training_data = [
    ("hello", "Greeting"),
    ("hi there", "Greeting"),
    ("hey", "Greeting"),
    ("greetings", "Greeting"),
    ("what's up", "Greeting"),
    ("goodbye", "Goodbye"),
    ("bye", "Goodbye"),
    ("see you later", "Goodbye"),
    ("what is water", "Water Definition"),
    ("define water", "Water Definition"),
    ("tell me about water", "Water Definition"),
    ("how much of the earth is covered by water", "Water Coverage on Earth"),
    ("what percentage of earth is water", "Water Coverage on Earth"),
    ("earth water coverage", "Water Coverage on Earth"),
    ("what are the states of water", "States of Water"),
    ("water in different forms", "States of Water"),
    ("solid liquid gas water", "States of Water"),
    ("explain the water cycle", "Water Cycle"),
    ("how does the water cycle work", "Water Cycle"),
    ("water cycle process", "Water Cycle"),
    ("why is clean water important", "Importance of Clean Water"),
    ("importance of clean drinking water", "Importance of Clean Water"),
    ("clean water health", "Importance of Clean Water"),
    ("What are the different types of water bodies", "Types of water bodies"),
    ("What are some regulations related to water pollution","regulations of water bodies"),
    ("What is the importance of conservation of water","conservation"),
    ("What is the pollution level of water","pollution level"),
    ("What are the most common pollutants in water","pollutant"),
    ("How can I get involved in water conservation","conservation"),
    ("What statistical analysis can be performed for water samples collected seasonally","statistical analysis"),
    ("How do researchers assess microplastic accumulation in water bodies? What steps are involved in this process?"," microplastic"),
    ("Why pH balance is important for fish culture"," pH balance"),
    ("What the possible Persistent Organic Pollutants and Heavy metals present in fluorspar, sediments, and water bodies around its mining area","pollutant"),
    (" How to improve quantity and quality of groundwater ","impoving water quality"),
    ("Is it viable to use hyperspectral remote sensing data to classify different types of water bodies in a region","hyperspetral remote sensing"),


for sentence, intent in training_data:
    print(f"Sentence: '{sentence}', Intent: '{intent}'")

Fitting a machine learning model with naive-bayes technique and creating a pipeline for Tf-IdfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

# Separate sentences and intents
sentences = [sentence for sentence, intent in training_data]
labels = [intent for sentence, intent in training_data]

# Create a pipeline for TF-IDF vectorization and Naive Bayes classification
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(sentences, labels)

print("Model training complete.")

Main chatbot logic is integrated below

In [7]:

import random
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline


nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)



try:
    with open('water.txt', 'r', errors='ignore') as f:
        raw = f.read().lower()
except FileNotFoundError:
    print("Error: water.txt not found. Please make sure the file exists in the correct directory.")
    raw = ""
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    raw = ""

sent_tokens = nltk.sent_tokenize(raw)

lemmer = nltk.stem.WordNetLemmatizer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

if sent_tokens:
    tfidf = TfidfVec.fit_transform(sent_tokens)
else:
    tfidf = None

def response(user_response):
    robo_response = ''

    if not sent_tokens or tfidf is None:
        return "I don't have information on that topic."

    temp_sent_tokens = sent_tokens + [user_response]

    user_tfidf = TfidfVec.transform([user_response])
    vals = cosine_similarity(user_tfidf, tfidf)

    idx = vals.argsort()[0][-1]

    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-1]

    if req_tfidf == 0:
        robo_response = "I am sorry! I don't understand you"
    else:
        robo_response = sent_tokens[idx]

    return robo_response


GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

intent_responses = {
    'Greeting': random.choice(GREETING_RESPONSES),
    'Goodbye': "Bye! take care..",
    'Water Definition': "Water is essential for life.",
    'Water Coverage on Earth': "It covers about 71% of the Earth's surface.",
    'States of Water': "Water exists in three states: solid, liquid, and gas.",
    'Water Cycle': "The water cycle is a continuous process.",
    'Importance of Clean Water': "Clean water is important for health.",
    # Add more specific responses for other intents as needed
    'Types of water bodies': "Examples of water bodies include oceans, lakes, rivers, and wetlands.",
    'regulations of water bodies': "Regulations vary by location, but often include rules on pollution discharge and water usage.",
    'conservation': "Water conservation involves using water efficiently to prevent waste. You can get involved by fixing leaks, using less water, and supporting conservation efforts.",
    'pollution level': "Water pollution levels are measured based on contaminants present. Specific levels depend on the location and type of water body.",
    'pollutant': "Common water pollutants include chemicals, waste, and microorganisms.",
    'statistical analysis': "Statistical analysis of water samples can involve examining trends in water quality, pollutant levels, and other parameters over time or across different locations.",
    'microplastic': "Researchers assess microplastic accumulation by collecting water samples, filtering them to isolate microplastics, and then analyzing the types and quantities present using microscopy or spectroscopy.",
    'pH balance': "Maintaining the correct pH balance in water is crucial for fish culture as it affects their physiological processes and overall health.",
    'impoving water quality': "Improving groundwater quality can involve reducing sources of contamination, implementing proper waste disposal, and using filtration or treatment methods.",
    'hyperspetral remote sensing': "Hyperspectral remote sensing can be a viable tool for classifying different water bodies based on their spectral signatures, which are influenced by factors like depth, turbidity, and the presence of algae or other substances."


}



flag = True
print("Aquabot: My name is golden_lotus. I will answer your queries regarding aqualogic.")
while(flag == True):

    training_data = [

        ("hello", "Greeting"),
        ("hi there", "Greeting"),
        ("hey", "Greeting"),
        ("greetings", "Greeting"),
        ("what's up", "Greeting"),
        ("goodbye", "Goodbye"),
        ("bye", "Goodbye"),
        ("see you later", "Goodbye"),
        ("what is water", "Water Definition"),
        ("define water", "Water Definition"),
        ("tell me about water", "Water Definition"),
        ("how much of the earth is covered by water", "Water Coverage on Earth"),
        ("what percentage of earth is water", "Water Coverage on Earth"),
        ("earth water coverage", "Water Coverage on Earth"),
        ("what are the states of water", "States of Water"),
        ("water in different forms", "States of Water"),
        ("solid liquid gas water", "States of Water"),
        ("explain the water cycle", "Water Cycle"),
        ("how does the water cycle work", "Water Cycle"),
        ("water cycle process", "Water Cycle"),
        ("why is clean water important", "Importance of Clean Water"),
        ("importance of clean drinking water", "Importance of Clean Water"),
        ("clean water health", "Importance of Clean Water"),
        ("What are the different types of water bodies", "Types of water bodies"),
        ("What are some regulations related to water pollution","regulations of water bodies"),
        ("What is the importance of conservation of water","conservation"),
        ("What is the pollution level of water","pollution level"),
        ("What are the most common pollutants in water","pollutant"),
        ("How can I get involved in water conservation","conservation"),
        ("What statistical analysis can be performed for water samples collected seasonally","statistical analysis"),
        ("How do researchers assess microplastic accumulation in water bodies? What steps are involved in this process?"," microplastic"),
        ("Why pH balance is important for fish culture"," pH balance"),
        ("What the possible Persistent Organic Pollutants and Heavy metals present in fluorspar, sediments, and water bodies around its mining area","pollutant"),
        (" How to improve quantity and quality of groundwater ","impoving water quality"),
        ("Is it viable to use hyperspectral remote sensing data to classify different types of water bodies in a region","hyperspetral remote sensing"),
        # Adding more diverse training data for water-related intents
        ("tell me about water pollution", "Water Definition"),
        ("what causes water pollution", "Water Definition"),
        ("types of water pollution", "Water Definition"),
        ("what are water bodies", "Types of water bodies"),
        ("examples of water bodies", "Examples of water bodies"),
        ("how to conserve water", "conservation"),
        ("tips for saving water", "conservation"),
        ("what are water pollution regulations", "regulations of water bodies"),
        ("laws about water pollution", "regulations of water bodies"),
        ("how is water pollution measured", "pollution level"),
        ("levels of pollutants in water", "pollution level"),
        ("what are pollutants in water", "pollutant"),
        ("chemicals in water", "pollutant"),
        ("heavy metals in water", "pollutant"),
        ("microplastics in the ocean", "microplastic"),
        ("how to measure microplastics in water", "microplastic"),
        ("why is pH important for fish", "pH balance"),
        ("ideal pH for fish", "pH balance"),
        ("how to improve groundwater", "impoving water quality"),
        ("cleaning groundwater", "impoving water quality"),
        ("using remote sensing for water bodies", "hyperspetral remote sensing"),
        ("classifying water bodies with satellite data", "hyperspetral remote sensing"),
    ]

    sentences = [sentence for sentence, intent in training_data]
    labels = [intent for sentence, intent in training_data]

    model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    model.fit(sentences, labels)

    user_response = input()
    user_response = user_response.lower()

    if user_response != 'bye':
        if user_response == 'thanks' or user_response == 'thank you':
            flag = False
            print("Aquabot: You are welcome..")
        else:

            predicted_intent = model.predict([user_response])[0]
            print(f" {predicted_intent}")
            if predicted_intent in intent_responses:
                print("Aquabot: " + intent_responses[predicted_intent])
            else:

                print("Aquabot:", end='')
                print(response(user_response))
    else:
        flag = False
        print("Aquabot: Bye! take care\n
        See you next time with more information and discussions")

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'u', 'wa'] not in stop_words.
  warnings.warn(


ROBO: My name is bot. I will answer your queries.
bye
ROBO: Bye! take care..
